In [1]:
# upload the dataset, count id stations complete for all years and save cleaned data

import pandas as pd
import numpy as np

# function to add week number to the data frame

def add_week(data_frame):
  data_frame['Time'] = pd.to_datetime(data_frame['Time'])
  data_frame['Week'] = data_frame['Time'].dt.isocalendar().week
  return data_frame

# drop NaN in pm25, add week and log_pm25 and keep only stations with complete pm25 values

file_list = ['dataset_2016.csv', 'dataset_2017.csv', 'dataset_2018.csv', 'dataset_2019.csv', 'dataset_2020.csv', 'dataset_2021.csv']
years = ['2016', '2017', '2018', '2019', '2020', '2021']
df_complete_id_stations = []

for file, year in zip(file_list, years):
  df = pd.read_csv(file)
  df.dropna(subset=['AQ_pm25'], inplace=True)
  station_counts = df['IDStations'].value_counts()
  max_occurrences = station_counts.max()
  stations_max_occurrences = station_counts[station_counts == max_occurrences].index.tolist()
  df = df[df['IDStations'].isin(stations_max_occurrences)]
  df['log_pm25'] = np.log(df['AQ_pm25']+1)
  df = add_week(df)
  name_file = f'dataset_{year}_cleaned.csv'
  df.to_csv(name_file, index=False)
  number_stations_max_occurrences = len(stations_max_occurrences)
  df_complete_id_stations.append(number_stations_max_occurrences)

print("Results:")
for idx, stations_52 in enumerate(df_complete_id_stations, 1):
    print(f"Dataset {idx}: Number of station id that appear all the weeks: {stations_52}")

# for dataset_2021_cleaned.csv week are wrong (it reads 1/1 as week 53) --> don't use it!

Results:
Dataset 1: Number of station id that appear all the weeks: 42
Dataset 2: Number of station id that appear all the weeks: 41
Dataset 3: Number of station id that appear all the weeks: 43
Dataset 4: Number of station id that appear all the weeks: 46
Dataset 5: Number of station id that appear all the weeks: 45
Dataset 6: Number of station id that appear all the weeks: 29


In [2]:
# we use year 2019

df = pd.read_csv('dataset_2019_cleaned.csv')
df

,IDStations,Latitude,Longitude,Time,Altitude,AQ_pm10,AQ_pm25,AQ_co,AQ_nh3,AQ_nox,...,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LA_hvi,LA_lvi,LA_land_use,LA_soil_use,log_pm25,Week
0,1264,46.167852,9.87921,2019-01-04,290,42.000000,31.714286,NaN,NaN,89.381429,...,10.568,2.6433,0.287400,4.647000,3.996143,1.232714,112,17.0,3.487812,1
1,1264,46.167852,9.87921,2019-01-11,290,33.428571,26.857143,NaN,NaN,89.350000,...,10.611,2.6905,0.287400,4.647000,3.994429,1.231000,112,17.0,3.327089,2
2,1264,46.167852,9.87921,2019-01-18,290,34.857143,28.714286,NaN,NaN,78.017143,...,10.650,2.7215,0.294243,4.647000,3.995000,1.231429,112,17.0,3.391628,3
3,1264,46.167852,9.87921,2019-01-25,290,42.857143,35.142857,NaN,NaN,89.498571,...,10.586,2.7023,0.335300,4.647000,3.996000,1.232714,112,17.0,3.587479,4
4,1264,46.167852,9.87921,2019-02-01,290,33.571429,30.428571,NaN,NaN,95.164286,...,10.317,2.5831,0.335300,4.647000,3.996857,1.234143,112,17.0,3.447717,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,STA.IT2282A,45.439700,8.62040,2019-11-29,154,24.000000,18.142857,NaN,NaN,NaN,...,72.580,11.2580,63.570000,7.836714,4.138286,1.009229,112,NaN,2.951930,48
2388,STA.IT2282A,45.439700,8.62040,2019-12-06,154,43.000000,33.857143,NaN,NaN,NaN,...,72.370,11.3220,63.570000,7.846714,4.138000,0.984329,112,NaN,3.551258,49
2389,STA.IT2282A,45.439700,8.62040,2019-12-13,154,28.571429,23.000000,NaN,NaN,NaN,...,70.484,11.1090,63.562857,7.852286,4.137571,0.962929,112,NaN,3.178054,50
2390,STA.IT2282A,45.439700,8.62040,2019-12-20,154,15.285714,8.714286,NaN,NaN,NaN,...,68.036,10.7940,63.560000,7.860000,4.136714,0.953657,112,NaN,2.273598,51


In [3]:
# find NaN values
missing_values = df.isnull().sum()

# print NaN number for every covariate
print(missing_values)

IDStations                        0
Latitude                          0
Longitude                         0
Time                              0
Altitude                          0
AQ_pm10                          54
AQ_pm25                           0
AQ_co                          1614
AQ_nh3                         2186
AQ_nox                          461
AQ_no2                           97
AQ_so2                         1728
WE_temp_2m                        0
WE_wind_speed_10m_mean            0
WE_wind_speed_10m_max             0
WE_mode_wind_direction_10m        0
WE_tot_precipitation              0
WE_precipitation_t                0
WE_surface_pressure               0
WE_solar_radiation                0
WE_rh_min                         0
WE_rh_mean                        0
WE_rh_max                         0
WE_wind_speed_100m_mean           0
WE_wind_speed_100m_max            0
WE_mode_wind_direction_100m       0
WE_blh_layer_max                  0
WE_blh_layer_min            

In [14]:
# drop from dataset covariates AQ_co, AQ_nh3, AQ_so2, LA_soil_use
covariates_to_drop = ['AQ_co', 'AQ_nh3', 'AQ_so2', 'LA_soil_use']

df = df.drop(columns=covariates_to_drop)
df

,IDStations,Latitude,Longitude,Time,Altitude,AQ_pm10,AQ_pm25,AQ_nox,AQ_no2,WE_temp_2m,...,EM_nox_traffic,EM_nox_sum,EM_so2_sum,LI_pigs,LI_bovine,LA_hvi,LA_lvi,LA_land_use,log_pm25,Week
0,1264,46.167852,9.87921,2019-01-04,290,42.000000,31.714286,89.381429,43.750000,-7.062714,...,0.599971,10.568,2.6433,0.287400,4.647000,3.996143,1.232714,112,3.487812,1
1,1264,46.167852,9.87921,2019-01-11,290,33.428571,26.857143,89.350000,39.615714,-6.936143,...,0.604714,10.611,2.6905,0.287400,4.647000,3.994429,1.231000,112,3.327089,2
2,1264,46.167852,9.87921,2019-01-18,290,34.857143,28.714286,78.017143,41.552857,-11.653429,...,0.615429,10.650,2.7215,0.294243,4.647000,3.995000,1.231429,112,3.391628,3
3,1264,46.167852,9.87921,2019-01-25,290,42.857143,35.142857,89.498571,44.395714,-9.844143,...,0.626857,10.586,2.7023,0.335300,4.647000,3.996000,1.232714,112,3.587479,4
4,1264,46.167852,9.87921,2019-02-01,290,33.571429,30.428571,95.164286,47.047143,-5.941429,...,0.634971,10.317,2.5831,0.335300,4.647000,3.996857,1.234143,112,3.447717,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2387,STA.IT2282A,45.439700,8.62040,2019-11-29,154,24.000000,18.142857,NaN,31.082857,5.795429,...,2.634143,72.580,11.2580,63.570000,7.836714,4.138286,1.009229,112,2.951930,48
2388,STA.IT2282A,45.439700,8.62040,2019-12-06,154,43.000000,33.857143,NaN,39.458571,3.981000,...,2.538571,72.370,11.3220,63.570000,7.846714,4.138000,0.984329,112,3.551258,49
2389,STA.IT2282A,45.439700,8.62040,2019-12-13,154,28.571429,23.000000,NaN,38.082857,5.991143,...,2.382429,70.484,11.1090,63.562857,7.852286,4.137571,0.962929,112,3.178054,50
2390,STA.IT2282A,45.439700,8.62040,2019-12-20,154,15.285714,8.714286,NaN,40.385714,6.512143,...,2.217429,68.036,10.7940,63.560000,7.860000,4.136714,0.953657,112,2.273598,51


In [33]:
# NaN values for stations
interested_covariates = ['AQ_pm10', 'AQ_nox', 'AQ_no2', 'LI_pigs', 'LI_bovine']
station_grouped = df.groupby('IDStations')
station_missing_values = {}

for station, df_station in station_grouped:
    print(f"Station: {station}")
    covariates_station = df_station[interested_covariates]
    missing_values = covariates_station.isnull().sum()
    covariate_with_missing = missing_values > 24
    if covariate_with_missing.any():
        station_missing_values[station] = covariate_with_missing[covariate_with_missing].index.tolist()
    print(missing_values)

# print stations with missing values
for station, covariate_list in station_missing_values.items():
    print(f"Station: {station} - Covariate with missing values: {covariate_list}")

Station: 1264
AQ_pm10      0
AQ_nox       0
AQ_no2       0
LI_pigs      0
LI_bovine    0
dtype: int64
Station: 1265
AQ_pm10      0
AQ_nox       0
AQ_no2       0
LI_pigs      0
LI_bovine    0
dtype: int64
Station: 1297
AQ_pm10      0
AQ_nox       0
AQ_no2       0
LI_pigs      0
LI_bovine    0
dtype: int64
Station: 548
AQ_pm10      1
AQ_nox       2
AQ_no2       1
LI_pigs      0
LI_bovine    0
dtype: int64
Station: 554
AQ_pm10      0
AQ_nox       0
AQ_no2       0
LI_pigs      0
LI_bovine    0
dtype: int64
Station: 560
AQ_pm10      0
AQ_nox       0
AQ_no2       0
LI_pigs      0
LI_bovine    0
dtype: int64
Station: 561
AQ_pm10      0
AQ_nox       0
AQ_no2       0
LI_pigs      0
LI_bovine    0
dtype: int64
Station: 576
AQ_pm10      0
AQ_nox       0
AQ_no2       0
LI_pigs      0
LI_bovine    0
dtype: int64
Station: 583
AQ_pm10      0
AQ_nox       0
AQ_no2       0
LI_pigs      0
LI_bovine    0
dtype: int64
Station: 592
AQ_pm10      0
AQ_nox       0
AQ_no2       0
LI_pigs      0
LI_bovine    0


In [34]:
# drop stations with too much missing values
IDStations_to_drop = list(station_missing_values.keys())
df_complete = df[~df['IDStations'].isin(IDStations_to_drop)]

# find NaN values
missing_values = df_complete.isnull().sum()

# print NaN number for every covariate
print(missing_values)

IDStations                      0
Latitude                        0
Longitude                       0
Time                            0
Altitude                        0
AQ_pm10                         2
AQ_pm25                         0
AQ_nox                         20
AQ_no2                         17
WE_temp_2m                      0
WE_wind_speed_10m_mean          0
WE_wind_speed_10m_max           0
WE_mode_wind_direction_10m      0
WE_tot_precipitation            0
WE_precipitation_t              0
WE_surface_pressure             0
WE_solar_radiation              0
WE_rh_min                       0
WE_rh_mean                      0
WE_rh_max                       0
WE_wind_speed_100m_mean         0
WE_wind_speed_100m_max          0
WE_mode_wind_direction_100m     0
WE_blh_layer_max                0
WE_blh_layer_min                0
EM_nh3_livestock_mm             0
EM_nh3_agr_soils                0
EM_nh3_agr_waste_burn           0
EM_nh3_sum                      0
EM_nox_traffic

In [87]:
# fill NaN in df_complete

nan_indices, nan_columns = np.where(pd.isnull(df_complete))
print(nan_indices)
print(nan_columns)

df_filled = df_complete.copy()
df_filled.iloc[187, [5, 7]] = (df_complete.iloc[186, [5, 7]] + df_complete.iloc[188, [5, 7]]) / 2
df_filled.iloc[203, [7, 8]] = (df_complete.iloc[202, [7, 8]] + df_complete.iloc[204, [7, 8]]) / 2
df_filled.iloc[944:953, [7, 8]] = (df_complete.iloc[943, [7, 8]] + df_complete.iloc[954, [7, 8]]) / 2
df_filled.iloc[967, [5, 7]] = (df_complete.iloc[966, [5, 7]] + df_complete.iloc[966, [5, 7]]) / 2
df_filled.iloc[1075:1077, [7, 8]] = (df_complete.iloc[1074, [7, 8]] + df_complete.iloc[1077, [7, 8]]) / 2
df_filled.iloc[1121:1123, [7, 8]] = (df_complete.iloc[1120, [7, 8]] + df_complete.iloc[1123, [7, 8]]) / 2
df_filled.iloc[1268:1270, [7, 8]] = (df_complete.iloc[1267, [7, 8]] + df_complete.iloc[1270, [7, 8]]) / 2
df_filled.iloc[1279, 7] = (df_complete.iloc[1278, 7] + df_complete.iloc[1280, 7]) / 2
df_filled.iloc[1282, [7, 8]] = (df_complete.iloc[1281, [7, 8]] + df_complete.iloc[1283, [7, 8]]) / 2

# find NaN values
missing_values = df_filled.isnull().sum()

# print NaN number for every covariate
print(missing_values)

[ 187  187  203  203  944  944  945  945  946  946  947  947  948  948
  949  949  950  950  951  951  952  952  967  967 1075 1075 1076 1076
 1121 1121 1122 1122 1268 1268 1269 1269 1279 1282 1282]
[5 7 7 8 7 8 7 8 7 8 7 8 7 8 7 8 7 8 7 8 7 8 5 7 7 8 7 8 7 8 7 8 7 8 7 8 7
 7 8]
IDStations                     0
Latitude                       0
Longitude                      0
Time                           0
Altitude                       0
AQ_pm10                        0
AQ_pm25                        0
AQ_nox                         0
AQ_no2                         0
WE_temp_2m                     0
WE_wind_speed_10m_mean         0
WE_wind_speed_10m_max          0
WE_mode_wind_direction_10m     0
WE_tot_precipitation           0
WE_precipitation_t             0
WE_surface_pressure            0
WE_solar_radiation             0
WE_rh_min                      0
WE_rh_mean                     0
WE_rh_max                      0
WE_wind_speed_100m_mean        0
WE_wind_speed_100m_max     

In [92]:
# create a function that from a data_frame return a dictionary with id_station and time series of log_pm25 values

def create_time_series(data_frame):
    stations = data_frame['IDStations'].unique()

    dict_stations = {}  # stations -> time series
    for station in stations:
        # Select data for current station
        data_station = data_frame[data_frame['IDStations'] == station]

        # Crea una time series per la stazione corrente
        log_pm25_series = data_station.set_index('Week')['log_pm25']

        # Aggiungi la time series al dizionario
        dict_stations[station] = log_pm25_series

    return dict_stations

dict_time_series = create_time_series(df_filled)
# access to a particular station --> dict_time_series['station_name']
# access to log_pm25 values --> dict_time_series['station_name'].values
print(dict_time_series['669'].index)
print(dict_time_series['669'].values)
# acces to log_pm25 values of a specific week (ex week 13) --> dict_time_series['station_name'][13]



Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
            18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
            35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
            52],
           dtype='int64', name='Week')
[3.76782266 3.75118334 3.69953675 3.78418963 3.64507683 3.61477148
 4.12713439 3.85318251 3.7809376  3.21887582 3.31678004 3.28519847
 3.31158522 3.12299405 3.12299405 2.8498804  2.6492097  2.62880083
 2.76362005 2.91235066 2.81341072 3.09104245 3.2054528  2.8678989
 2.7080502  2.95936463 2.76362005 2.62708114 3.2470467  2.94443898
 2.61843804 2.66921037 2.43611649 2.89037176 2.57768838 2.68881884
 3.21887582 3.12299405 3.21887582 3.07269331 3.61091791 3.10608033
 2.94443898 2.81341072 2.78147767 2.89827694 2.71752895 3.36235755
 3.71008166 3.74950408 3.14271446 3.87713575]


In [93]:
def create_matrix_from_dict(dict_stations):
    # Obtain number of stations and max length of time series
    num_stations = len(dict_stations)
    max_length = max(len(series) for series in dict_stations.values())

    # Initialize matrix with NaN values
    matrix = np.full((num_stations, max_length), np.nan)

    # Put log_pm25 values into matrix
    for idx, series in enumerate(dict_stations.values()):
        matrix[idx, :len(series)] = series.values

    return matrix

log_pm25_matrix = create_matrix_from_dict(dict_time_series)

print(log_pm25_matrix)
print(log_pm25_matrix.shape)

[[3.48781185 3.32708941 3.39162793 ... 3.64133851 3.19575341 3.63381968]
 [3.88890059 3.52636052 3.74106521 ... 3.48343548 2.7080502  4.05797692]
 [4.05550473 3.79066215 3.83483337 ... 3.63758616 3.05803616 4.15888308]
 ...
 [3.74444931 3.61861026 3.40594798 ... 3.39162793 2.61843804 3.81928095]
 [3.73766962 3.45676723 3.37220984 ... 3.42006587 2.56494936 3.79388276]
 [3.9539872  3.74782199 3.07136969 ... 3.2358734  2.6492097  3.68887945]]
(34, 52)
